## Context Relevance Feedback Requirements
1. Relevance requires adherence to the entire query.
2. Long context with small relevant chunks are relevant.
3. Context that provides no answer can still be relevant.
4. Feedback mechanism should differentiate between seeming and actual relevance.
5. Relevant but inconclusive statements should get increasingly high scores as they are more helpful for answering the query.

Below are examples of usage. Find the [results](#test-results) of the tests tabulated at the bottom.

In [ ]:
# Import relevance feedback function
from trulens_eval.feedback import GroundTruthAgreement, OpenAI as fOpenAI
from trulens_eval import TruBasicApp, Feedback, Tru, Select
from test_cases import context_relevance_golden_set
fopenai = fOpenAI()

import openai

Tru().reset_database()

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "..."

In [3]:
turbo = fOpenAI(model_engine="gpt-3.5-turbo")
# Define your feedback functions
def wrapped_relevance_turbo(input, output):
    return turbo.qs_relevance(input, output)

# Define your feedback functions
def wrapped_relevance_with_cot_turbo(input, output):
    return turbo.qs_relevance_with_cot_reasons(input, output)

gpt4 = fOpenAI(model_engine="gpt-4")
# Define your feedback functions
def wrapped_relevance_gpt4(input, output):
    return gpt4.qs_relevance(input, output)

# Define your feedback functions
def wrapped_relevance_with_cot_gpt4(input, output):
    return gpt4.qs_relevance_with_cot_reasons(input, output)

In [4]:
# Create a Feedback object using the numeric_difference method of the ground_truth object
ground_truth = GroundTruthAgreement(context_relevance_golden_set)
# Call the numeric_difference method with app and record
f_groundtruth = Feedback(ground_truth.numeric_difference, name = "Context Relevance Smoke Test").on(Select.Record.calls[0].args.args[0]).on(Select.Record.calls[0].args.args[1]).on_output()

✅ In Context Relevance Smoke Test, input prompt will be set to *.__record__.calls[0].args.args[0] .
✅ In Context Relevance Smoke Test, input response will be set to *.__record__.calls[0].args.args[1] .
✅ In Context Relevance Smoke Test, input score will be set to *.__record__.main_output or `Select.RecordOutput` .


In [5]:
tru_wrapped_relevance_turbo = TruBasicApp(wrapped_relevance_turbo, app_id = "context relevance gpt-3.5-turbo", feedbacks=[f_groundtruth])
tru_wrapped_relevance_with_cot_turbo = TruBasicApp(wrapped_relevance_with_cot_turbo, app_id = "context relevance with cot reasoning gpt-3.5-turbo", feedbacks=[f_groundtruth])
tru_wrapped_relevance_gpt4 = TruBasicApp(wrapped_relevance_gpt4, app_id = "context relevance gpt-4", feedbacks=[f_groundtruth])
tru_wrapped_relevance_with_cot_gpt4 = TruBasicApp(wrapped_relevance_with_cot_gpt4, app_id = "context relevance with cot reasoning gpt-4", feedbacks=[f_groundtruth])

In [6]:
for i in range(len(context_relevance_golden_set)):
    prompt = context_relevance_golden_set[i]["query"]
    response = context_relevance_golden_set[i]["response"]
    tru_wrapped_relevance_turbo.call_with_record(prompt, response)
    tru_wrapped_relevance_with_cot_turbo.call_with_record(prompt, response)
    tru_wrapped_relevance_gpt4.call_with_record(prompt, response)
    tru_wrapped_relevance_with_cot_gpt4.call_with_record(prompt, response)

Task queue full. Finishing existing tasks.


In [8]:
Tru().get_records_and_feedback(app_ids=[])[0].groupby('app_id')[['Context Relevance Smoke Test', 'latency', 'total_cost']].mean()

,Context Relevance Smoke Test,latency,total_cost
app_id,,,
context relevance gpt-3.5-turbo,0.800000,0.066667,0.000762
context relevance gpt-4,0.786667,0.066667,0.015268
context relevance with cot reasoning gpt-3.5-turbo,0.726667,0.066667,0.000917
context relevance with cot reasoning gpt-4,0.766667,0.066667,0.019316
